In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
from collections import Counter
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sign-language-image-classification/submit.csv
/kaggle/input/sign-language-image-classification/train.csv
/kaggle/input/sign-language-image-classification/test.csv
/kaggle/input/sign-language-image-classification/images/images/test/91ad97d64ba1a9e4d77f960bdae6efd3.jpg
/kaggle/input/sign-language-image-classification/images/images/test/81cf1d5b3af487cc91cb7facfdea24db.jpg
/kaggle/input/sign-language-image-classification/images/images/test/ffa44bfbce1fdecd9302524a6cd9bb37.jpg
/kaggle/input/sign-language-image-classification/images/images/test/9188ea7efca3896bae639805dc636d03.jpg
/kaggle/input/sign-language-image-classification/images/images/test/ceaa79ac54b72590b41a0105176c455c.jpg
/kaggle/input/sign-language-image-classification/images/images/test/51e9c7f5620ed968f9c74b102fe435c1.jpg
/kaggle/input/sign-language-image-classification/images/images/test/2493551f1dfb19a3a71104c89546d0f2.jpg
/kaggle/input/sign-language-image-classification/images/images/test/3c07e8a3939412b5f93c

In [2]:
working_dir=os.getcwd()
print(working_dir)

/kaggle/working


In [3]:
base_dir='/kaggle/input/sign-language-image-classification'
print(os.path.isdir(base_dir))

True


In [4]:
my_trainpath='/kaggle/working/my_train'
my_testpath='/kaggle/working/my_test'

In [5]:
for i in range(20):
    os.makedirs(os.path.join(working_dir,f'my_test/{i}'),exist_ok=True)
for i in range(20):
    os.makedirs(os.path.join(working_dir,f'my_train/{i}'),exist_ok=True)

In [6]:
train_csv_path=os.path.join(base_dir,'train.csv')
test_csv_path=os.path.join(base_dir,'test.csv')

train_csv=pd.read_csv(train_csv_path)
test_csv=pd.read_csv(test_csv_path)
train_csv.head(10)

,id,label
0,b1627fb5d6af156b41c46704bc041416.jpg,8
1,bf61d61ec5f282372b76311e2594b94f.jpg,13
2,1e963951fb29f4f37d9a153e14533d80.jpg,17
3,4aec4a36181bd02af6040e3b4eef8378.jpg,14
4,78740bc6ad7b6ca6fe572a580ffa65e0.jpg,16
5,4caeafa514ae949eeeea54a71efd89c0.jpg,9
6,c2063e258414f2f5ec511fda06288621.jpg,3
7,cc34f29df0da071c3a8523b8d9478cbb.jpg,4
8,c620c2b977426bb6122dd834846aebd6.jpg,19
9,98f72a118aa838cd356d43d9546fcd39.jpg,18


In [7]:
test_csv.head(10)

,id
0,5f3540c43f86dab2c9a7c87834311ee3.jpg
1,7f14db51f1994bce42972c7fe111f5cd.jpg
2,092908960a3d4dccbe06a60b71dcdd98.jpg
3,542ed4f107f9472457b47a436ec55e60.jpg
4,c125af0d467cffe584728aad4b426d53.jpg
5,dac637220376b2f834447c515148e301.jpg
6,c94daf6d789e53fd933cb26cedf00678.jpg
7,559cbbb343a27ec2839477c53b0ba65b.jpg
8,4a6bd7247218fe3cd2001a634b1fd6df.jpg
9,0598c5bd6d54bb7d9e3b7ece979a118c.jpg


In [8]:
dict_data=train_csv.to_dict(orient='list')
label=dict_data['label']
id_data=dict_data['id']

In [9]:
np.max(dict_data['label'])

19

In [10]:
dict_test_data=test_csv.to_dict(orient='list')
id_data_test=dict_test_data['id']

In [11]:
train_path='/kaggle/input/sign-language-image-classification/images/images/train'
test_path='/kaggle/input/sign-language-image-classification/images/images/test'
print(len(os.listdir(test_path)))

1504


In [12]:
for i in range(2000):
    if i<=int(2000*0.3):
        shutil.copy(os.path.join(train_path,id_data[i]),os.path.join(my_testpath,str(label[i])))
    else:    
        shutil.copy(os.path.join(train_path,id_data[i]),os.path.join(my_trainpath,str(label[i])))

In [13]:
print(len(os.listdir('/kaggle/working/my_test/12')))

28


In [14]:
train_generator=ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='reflect',
)

In [15]:
validate_generator=ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='reflect',
)

In [16]:
train_generator=train_generator.flow_from_directory(
    my_trainpath,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=25
)

Found 1399 images belonging to 20 classes.


In [17]:
train_generator=validate_generator.flow_from_directory(
    my_testpath,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=25
)

Found 601 images belonging to 20 classes.


In [18]:
from tensorflow.keras.applications import InceptionV3

In [19]:
convet=InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
